# Regression with Orbit - Basic

In this demo, we want to demonstartate how to use the different arguments in the model classes (LGT or DLT) to realize different setups for the regressors. Those could be very useful in practice when tuning the models.

In [1]:
import pandas as pd
import numpy as np
from orbit.lgt import LGT
from orbit.utils.plot import plot_predicted_data
from orbit.utils.plot import plot_predicted_components

## load data

In [2]:
DATA_FILE = "./data/iclaims_example.csv"
df = pd.read_csv(DATA_FILE, parse_dates=['week'])
df.dtypes

week              datetime64[ns]
claims                     int64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
dtype: object

In [3]:
df.head(5)

,week,claims,trend.unemploy,trend.filling,trend.job
0,2010-01-03,651215,1.183973,0.720140,1.119669
1,2010-01-10,825891,1.183973,0.814896,1.178599
2,2010-01-17,659173,1.203382,0.739091,1.119669
3,2010-01-24,507651,1.164564,0.814896,1.107883
4,2010-01-31,538617,1.086926,0.776993,1.072525


## Use regressors and specify their signs

The regressor columns can be supplied via argument `regressor_col`. Their signs can be specified via `regressor_sign`, with values either '=' (regular, no restriction) or '+' (positive). These two lists should be of the same lenght. The default values of `regressor_sign` is all '='.

Also, note that in general, a better performance can be acheived in regressions with `sample_method=mcmc` due to potential high dimensional distributions of parameters. We will use `mcmc` as sample method in following examples.

In [4]:
DATE_COL="week"
RESPONSE_COL="claims"
REGRESSOR_COL=['trend.unemploy', 'trend.filling', 'trend.job']

In [5]:
lgt_mod=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='], 
    seasonality=52,
    seed=1,
    predict_method='median',
    sample_method='mcmc',
    is_multiplicative=True,
    auto_scale=False
)
lgt_mod.fit(df=df)
# for now it is a necessary step to call `.predict()` before getting posteriors
_ = lgt_mod.predict(df=df)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


The estimated regressor coefficients can be retrieved via `.aggregated_posteriors`.

In [6]:
lgt_mod.get_regression_coefs()

,regressor,regressor_sign,coefficient
0,trend.unemploy,Positive,0.052030
1,trend.filling,Positive,0.068881
2,trend.job,Regular,-0.072293


## Adjust pirors for regressor beta and regressor standard deviation

In the model, it is assumed $$\beta \sim Gaussian(\beta_{prior}, \sigma_{prior})$$

The default values for $\beta_{prior}$ and $\sigma_{prior}$ are 0 and 1, respectively.

Users could adjust them via arguments `regressor_beta_prior` and `regressor_sigma_prior`. These two lists should be of the same lenght as `regressor_col`.

In [7]:
lgt_mod=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='], 
    regressor_beta_prior=[0.05, 0.05, 0],
    regressor_sigma_prior=[0.1, 0.1, 0.1],
    seasonality=52,
    seed=1,
    predict_method='median',
    sample_method='mcmc',
    is_multiplicative=True
)

In [8]:
lgt_mod.fit(df=df)
_ = lgt_mod.predict(df=df)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


One can notice the significant changes in the estimted coefficients by using different priors.

In [9]:
lgt_mod.aggregated_posteriors['median']['pr_beta']

array([[0.0520675 , 0.06470291]])

In [10]:
lgt_mod.aggregated_posteriors['median']['rr_beta']

array([[-0.05638691]])

## Use data-driven sigma for each coefficients

Instead of using fixed standard deviations for regressors, a hyperprior can be assigned to them, i.e.
$$\sigma_\beta \sim \text{Half-Cauchy}(0, \text{ridge_scale})$$

This can be done by setting `regression_penalty="auto_ridge"`.  Notice there is hyperprior `auto_ridge_scale` for tuning with a default of `0.1`.

In [11]:
lgt_mod=LGT(
    response_col=RESPONSE_COL,
    date_col=DATE_COL,
    regressor_col=REGRESSOR_COL,
    regressor_sign=["+", '+', '='],
    seasonality=52,
    seed=1,
    predict_method='median',
    sample_method='mcmc',
    is_multiplicative=True,
    regression_penalty="auto_ridge",
    auto_ridge_scale=0.1,
)
lgt_mod.fit(df=df)
_ = lgt_mod.predict(df=df)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [12]:
lgt_mod.aggregated_posteriors['median']['pr_beta']

array([[0.04531444, 0.04043214]])

In [13]:
lgt_mod.aggregated_posteriors['median']['rr_beta']

array([[-0.03504845]])